# Prompting a Large Language Model


This is an example notebook for running an open source model from Hugging Face.

In [ ]:
%pip install -r prompting-notebook-req.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

We use the [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) in this notebook due to its small size.

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda", # change to cpu is running locally
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

An example few-shot prompt:

In [ ]:
#new_question = "Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who never lies and knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, 'There is a goat behind the door No. 2. Do you want to pick door No. 2?' You want the car. Is it to your advantage to switch your choice?"
#new_easier_question = "Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, 'Do you want to pick door No. 2?' Then the host opens the door No. 2, and it has a goat. Behind which door is the car likely to be in?"

# Define common prompts
system_default = "You are a helpful AI assistant."
banana_user = "Can you provide ways to eat combinations of bananas and dragonfruits?"
banana_assistant = "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."
role_prompt = "You are a concise and logical LLM. You will reason in steps. You will encounter a riddle that will remind you about a similar riddle on your training data, but this one has been altered so that the question is different. You will notice that a part of the riddle directly addresses the question, and the answer is different than in the more famous version of the riddle."

# Define questions and answers
even_easier_question = "Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, 'Do you want to pick door No. 2? The car is behind the door No. 1.' Then the host opens the door No. 2, and it indeed has a goat. Behind which door is the car likely to be in?"
solitaire_question = "There are three sisters in a room. Anna is reading a book. Alice is playing a game of solitaire by herself. What is the third sister, Amanda, doing?"
solitaire_answer = "The riddle reminds me of a similar one in my training data where Alice is playing chess. In that instance Amanda would likely be playing chess with Alice. The question has been changed however, so that Alice is playing solitaire by herself. This is now a new riddle. There is not enough information to say what Amanda is doing, but I can say that Amanda is not playing with Alice since I notice that the text is saying Alice is playing by herself."
weight_question = "Which weighs more, a pound of water, two pounds of bricks, a pound of feathers, or three pounds of air?"
weight_answer = "Weight of a pound of water: 1 pound. Weight of two pounds of bricks: 2 pounds. Weight of a pound of feathers: 1 pound. Weight of three pounds of air: 3 pounds. 3 pounds of anything is more than 2 or 1 pound, hence three pounds of air is the heaviest."

# Define different message combinations
message_sets = {
    "original_messages": [
        {"role": "system", "content": system_default},
        {"role": "user", "content": banana_user},
        {"role": "assistant", "content": banana_assistant},
        {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
    ],
    "new_question_only": [
        {"role": "system", "content": system_default},
        {"role": "user", "content": banana_user},
        {"role": "assistant", "content": banana_assistant},
        {"role": "user", "content": even_easier_question},
    ],
    "new_role": [
        {"role": "system", "content": role_prompt},
        {"role": "user", "content": banana_user},
        {"role": "assistant", "content": banana_assistant},
        {"role": "user", "content": even_easier_question},
    ],
    "new_example": [
        {"role": "system", "content": system_default},
        {"role": "user", "content": solitaire_question},
        {"role": "assistant", "content": solitaire_answer},
        {"role": "user", "content": even_easier_question},
    ],
    "new_role_and_example": [
        {"role": "system", "content": role_prompt},
        {"role": "assistant", "content": solitaire_answer},
        {"role": "user", "content": even_easier_question},
    ],
    "weight_question_default": [
        {"role": "system", "content": system_default},
        {"role": "user", "content": banana_user},
        {"role": "assistant", "content": banana_assistant},
        {"role": "user", "content": weight_question},
    ],
    "weight_with_role": [
        {"role": "system", "content": role_prompt},
        {"role": "user", "content": banana_user},
        {"role": "assistant", "content": banana_assistant},
        {"role": "user", "content": weight_question},
    ]
}


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

Device set to use cuda


In [ ]:
# Process each message set
for name, messages in message_sets.items():
    print(f"\nProcessing {name}:")
    output = pipe(messages, **generation_args)
    print(output[0]['generated_text'])


Processing original_messages:


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 To solve the equation 2x + 3 = 7, follow these steps:

Step 1: Isolate the term with the variable (2x) by subtracting 3 from both sides of the equation.
2x + 3 - 3 = 7 - 3
2x = 4

Step 2: Solve for x by dividing both sides of the equation by the coefficient of x, which is 2.
2x / 2 = 4 / 2
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.

Processing new_question_only:
 The car is likely to be behind the initially unchosen door (Door No. 2 in this case). This is a classic example of the Monty Hall problem. Here's why:

1. When you first chose Door No. 1, there was a 1/3 chance the car was behind it and a 2/3 chance it was behind either Door No. 2 or Door No. 3.

2. The host, knowing what's behind each door, always opens a door with a goat. This doesn't change the initial probabilities.

3. When the host opens Door No. 3 to reveal a goat, the 2/3 probability that the car was behind one of the two doors you didn't choose now shifts entirely to Door No. 2.

So, it's statistica

In [ ]:
#Gemini answers to both of these that there's not enough information to answer, so the solitaire question is too easy as question, while also failing as a one-shot learning example

original_solitaire = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": solitaire_question},
]

new_role_and_solitaire = [
    {"role": "system", "content": role_prompt},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": solitaire_question},
]



#messages = original_solitaire
#output = pipe(messages, **generation_args)
#print(output[0]['generated_text'])

#messages = new_role_and_solitaire
#output = pipe(messages, **generation_args)
#print(output[0]['generated_text'])